# Working with the 6-hourly Global Drifter Program (GDP) data

In this Notebook, we use the [6-hourly data](https://www.aoml.noaa.gov/phod/gdp/interpolated/data/all.php) from the [NOAA Global Drifter Program](https://www.aoml.noaa.gov/global-drifter-program/) to illustrate the steps necessary to process a Lagrangian dataset into a `RaggedArray` class object. This object can subsequently be converted to `xarray.Dataset` or `awkward.Array` instances for analysis. We also show how the *RaggedArray* object can also be written into files for archival.

## Import the `RaggedArray` class

In [ ]:
from clouddrift import RaggedArray

## Dataset-specific functions

The `gdp6h.py` module contains a number of specific functions for the current GDP 6-hourly files, including:

- `gdp6h.preprocess`: Preprocess and return an `xarray.Dataset` for a specific trajectory. 
- `gdp6h.download`: Download NetCDF files from the GDP FTP server.
- `gdp6h.rowsize [Optional]`: Return the dimension of a specific trajectory to speed up the preprocessing.

The `gdp6h` module is located in the `data/` directory located in the root of this repository, so we first need to add it to the search path:

In [ ]:
import sys
sys.path.append('../')
from data import gdp6h

## Download the GDP data

The `gdp6h.download` function will store the raw dataset into the `data/raw/gdp-6hourly/` folder (specified in the `gdp6h.py` module). By default `download()` will download the GPD dataset available on the [FTP/HTTPS repository](https://www.aoml.noaa.gov/ftp/pub/phod/lumpkin/netcdf/)).

In [ ]:
help(gdp6h.download)

With this function, it is also possible to retrieve a subset from a `drifter_ids` list or specified an integer `n_random_id` to randomly retrieve `n` trajectory. If both arguments are given, the function downloads `n_random_id` out of the list `drifter_ids`. The function returns the list of `drifters_ids` that was downloaded, and can be passed to create the ragged array.

In [ ]:
drifter_ids = gdp6h.download(n_random_id=100)

In [ ]:
drifter_ids[:5]

## Create the `RaggedArray` instance

Once the data are downloaded, the *RaggedArray* object can be created and archived as a NetCDF file or a parquet file. In addition, the ragged array can be converted to an [xarray Dataset](https://docs.xarray.dev/en/stable/generated/xarray.Dataset.html) or an [Awkward Array](https://awkward-array.readthedocs.io/en/stable/_auto/ak.Array.html) in order to conduct analyses.

Before we do that, we must first define the dataset-specific coordinates, data, and metadata information.

In [ ]:
coords = {
    'ids': 'ids',
    'time': 'time',
    'lon': 'longitude', # or alternatively lon360
    'lat': 'latitude'
}
data = ['ve', 'vn', 'temp', 'err_lat', 'err_lon', 'err_temp', 'drogue_status']
metadata = [
    'ID',
    'rowsize',
    'WMO',
    'expno',
    'deploy_date',
    'deploy_lat',
    'deploy_lon',
    'end_date',
    'end_lat',
    'end_lon',
    'start_date',
    'start_lat',
    'start_lon',
    'drogue_lost_date',
    'typedeath',
    'typebuoy',
    'DeployingShip',
    'DeploymentStatus',
    'BuoyTypeManufacturer',
    'BuoyTypeSensorArray',
    'CurrentProgram',
    'PurchaserFunding',
    'SensorUpgrade',
    'Transmissions',
    'DeployingCountry',
    'DeploymentComments',
    'ManufactureYear',
    'ManufactureMonth',
    'ManufactureSensorType',
    'ManufactureVoltage',
    'FloatDiameter',
    'SubsfcFloatPresence',
    'DrogueType',
    'DrogueLength',
    'DrogueBallast',
    'DragAreaAboveDrogue',
    'DragAreaOfDrogue',
    'DragAreaRatio',
    'DrogueCenterDepth',
    'DrogueDetectSensor'
]

Now we can pass this information together with the dataset-specific pre-processing function to process the GDP dataset from its own format into a `RaggedArray` instance:

In [ ]:
ra = RaggedArray.from_files(
    drifter_ids,
    gdp6h.preprocess,
    coords, 
    metadata,
    data,
    rowsize_func=gdp6h.rowsize
)

## Export to data files

Currently, exporting the `RaggedArray` instance to NetCDF and Apache Parquet file formats is supported.

### NetCDF

In [ ]:
ra.to_netcdf('../data/process/gdp_6h.nc')

### Apache Parquet

In [ ]:
ra.to_parquet('../data/process/gdp_6h.parquet')

## Import from data files

As with exporting to files, we can import a `RaggedArray` instance from a NetCDF or Apache Parquet files. 

### NetCDF

In [ ]:
ra = RaggedArray.from_netcdf('../data/process/gdp_6h.nc')

### Apache Parquet

In [ ]:
ra = RaggedArray.from_parquet('../data/process/gdp_6h.parquet')

## Converting to data structures for analysis

Typically, you would not do your data analysis directly on a `RaggedArray` instance.
Instead, you first convert it to a data structure that is geared for data analysis and exploratory science.
Currently, the library supports Xarray Dataset and Awkward Array structures.

### Xarray Dataset

In [ ]:
ds = ra.to_xarray()

Now we have our GDP dataset as an Xarray Dataset while also being optimized for memory.
Let's see what's inside:

In [ ]:
ds


Time, longitude, and latitude arrays are 1-dimensional and are effectively a concatenated list of drifter trajectories. The `ids` array can be used to mask (subset) a specific drifter. Let's make a quick trajectory map by looping over all the trajectories and plotting them one by one.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
for drifter_id in drifter_ids:
    this_drifter = ds.ids == drifter_id
    plt.plot(ds.lon[this_drifter], ds.lat[this_drifter], linestyle='', marker='.', ms=0.1)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('100 random trajectories from the 6-hourly GDP dataset')

You may find that the method above is relatively slow to plot for a large number of trajectories. Alternatively, we can construct an index variable that indexes the beginning of each trajectory in the ragged arrays (with index `0` for the first trajectory). We first need to import the `numpy` package to conduct some mathematical operations to calculate that index variable.

In [ ]:
import numpy as np

In [ ]:
traj_idx = np.insert(np.cumsum(ds.rowsize.values),0,0)

In [ ]:
for j in range(len(drifter_ids)):
    plt.plot(ds.lon[slice(traj_idx[j],traj_idx[j+1])], ds.lat[slice(traj_idx[j],traj_idx[j+1])], linestyle='', marker='.', ms=0.1)    
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('100 random trajectories from the 6-hourly GDP dataset')

### Awkward Array

In [ ]:
ds = ra.to_awkward()

With an awkward array, there is no need to define an index variable as for an xarray Dataset:

In [ ]:
for n in range(len(drifter_ids)):
    plt.plot(ds.obs['lon'][n], ds.obs['lat'][n], linestyle='', marker='.', ms=0.1)
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('100 random trajectories from a GDP dataset')